# pdb使用说明

pdb是一个Python内置的标准库，是一个Python调试工具。pdb提供了交互式命令行源码调试功能，包括如下主要特性：
- 断点设置
- 单步调试
- 进入函数
- 查看代码
- 查看栈片段
- 动态改变变量

pdb提供了一些常用命令，如下表所示，后续将逐一使用。

| 命令 | 说明  |
|:-----|:------|
| `break`, `b` | 设置断点 |
| `tbreak`| 临时设置断点 |
| `conditon`| 设置条件断点 |
| `clear`, `cl` | 清除断点 |
| `until`| 持续执行直到运行到指定行 |
| `jump`, `j`| 持续执行直到运行到指定行 |
| `list`, `l` | 查看当前行的代码块 |
| `longlist`, `ll` | 查看当前行的更多代码块 |
| `next`, `n` | 执行下一行 |
| `step`, `s` | 进入函数 |
| `reurn`, `r` | 执行代码知道从当前函数返回 |
| `exit`, `quit`, `q` | 终止并退出 |
| `print`, `p`| 打印变量 |
| `pp`| 打印变量的值 |

## 启动pdb

使用pdb工具可以把高速运行的Python程序慢下来，像放电影一样，自己来控制程序的运行。

下面先编写一个能够正常运行的Python程序：

In [1]:
%%writefile debug_demo01.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Copyright (c) 2018--2023, Wang Weihua.
# All rights reserved.
"""pdb demo program"""


class SimpleClass(object):
    """simple class"""
    def __init__(self):
        self.number = 0

    def increment(self):
        """increment 1"""
        self.number += 1


def afunc(var):
    """a function"""
    result = 0
    print(type(var), id(var), var)
    if isinstance(var, int):
        for i in range(var):
            result += i

    return result


def main():
    """main entry"""
    result = afunc(5)
    sa = SimpleClass()
    sa.increment()
    print(result, sa.number)


if __name__ == '__main__':
    main()


Overwriting debug_demo01.py


在命令行下，输入如下命令就进入调试状态：
```
python -m pdb debug_demo01.py
```

如果运行成功，结果如图所示：
```
> /home/whwang/training/python-expert/debug/debug_demo01.py(5)<module>()
-> """pdb demo program"""
(Pdb)
```

箭头`->`所指内容，为准备运行的当前语句：
```
-> """pdb demo program"""
```

下一行是pdb的提示符为`(Pdb)`。在该提示符下，可以输入pdb命令。

键入`help`命令并回车，会列出pdb常用命令：
```
(Pdb) help

Documented commands (type help <topic>):
========================================
EOF    c          d        h         list      q        rv       undisplay
a      cl         debug    help      ll        quit     s        unt
alias  clear      disable  ignore    longlist  r        source   until
args   commands   display  interact  n         restart  step     up
b      condition  down     j         next      return   tbreak   w
break  cont       enable   jump      p         retval   u        whatis
bt     continue   exit     l         pp        run      unalias  where

Miscellaneous help topics:
==========================
exec  pdb

```

执行`exit`命令，会终止并退出软件调试。

## 单步运行和查看

Python程序按照顺序自上而下运行，跳过注释行，首先来到文档字符串语句。


使用`list`命令会列出当前语句前后的语句块，同时会用箭头`->`来提醒当前语句；使用`ll`会列出更多代码块。

使用`next`命令会执行当前语句，并来到下一行语句，注意会跳过注释行以及空行。

执行`next`语句时，会一下子跳过`def`与`class`定义语句块。也就是说，定义语句块只是定义，并没有调用其中的语句，只要语法正确是不会出错的。

运行到`if`语句
```
(Pdb) l
 32         sa = SimpleClass()
 33         sa.increment()
 34         print(result, sa.number)
 35
 36
 37  -> if __name__ == '__main__':
 38         main()
[EOF]
```
再运行`next`命令时，会根据条件判断结果，跳到分支语句块中。

pdb控制程序来到
```
(Pdb) n
> d:\training\debug_demo01.py(38)<module>()
-> main()
(Pdb)
```

在调用函数`main()`时，可以做如下选择：
- 如果使用命令`next`，会把当前语句执行完毕，进入下一行；
- 如果使用命令`step`，会进入`main`函数空间。

这里使用命令`step`，并使用`list`命令查看：
```
(Pdb) l
 24                 result += i
 25
 26         return result
 27
 28
 29  -> def main():
 30         """main entry"""
 31         result = afunc(5)
 32         sa = SimpleClass()
 33         sa.increment()
 34         print(result, sa.number)
(Pdb)
```

会发现，pdb又来到了`main`函数定义语句，与上次不同的是，继续使用`next`命令，会进入到`main`函数的第一行语句。
```
(Pdb) n
> d:\training\debug_demo01.py(31)main()
-> result = afunc(5)
```

同样可以继续使用`step`命令，进入`afunc()`函数空间内部，继续进行pdb调试。使用`next`命令会继续单步执行下去，来到`for`循环语句：
```
(Pdb) next
> d:\training\debug_demo01.py(23)afunc()
-> for i in range(var):
```

在循环语句，使用`next`命令会反复运行循环体内的语句，可以使用`p`显示变量值。

如果认为函数体内没有什么问题，可以使用命令`return`，执行函数体内余下语句，并从当前函数返回。

来到构建对象语句：
```
(Pdb) n
> d:\training\debug_demo01.py(32)main()
-> sa = SimpleClass()
(Pdb) l
 27
 28
 29     def main():
 30         """main entry"""
 31         result = afunc(5)
 32  ->     sa = SimpleClass()
 33         sa.increment()
 34         print(result, sa.number)
 35
 36
 37     if __name__ == '__main__':
(Pdb)
```

同样可以用命令`next`或`step`，选择执行或进入类构造方法。

使用`step`命令，返回如下：
```
(Pdb) s
--Call--
> d:\training\debug_demo01.py(10)__init__()
-> def __init__(self):
(Pdb)     
```

最后，使用命令`exit`退出调试。

如上所示，使用pdb调试工具可以让Python程序慢下来，时停时走，间或查看语句执行结果，像会看电影一样，把程序的运行过程看了个清清楚楚明明白白。

## 飞跳运行

上一节，使用`next`或`step`等命令，逐行调试Python程序语句，使得我们能深入理解程序运行过程。然而，在调试程序定位Bug时，常常需要加快运行速度，一步到达指定位置。可以通过设置断点或通过指定语句行数来实现。

启动pdb调试器后，会来到程序的第一行语句。现在打算直接跳到第21行语句：
```
    print(type(var), id(var), var)
```

可以通过设置pdb命令`break`设置断点，其使用语法包括如下：
- `break lineno`，在指定行设置断点；
- `break filename:lineno` ，在指定文件指定行设置断点；
- `break functionname`，在指定函数名的第一行语句设置断点
```

这里使用如下命令：
```
(Pdb) break 21
Breakpoint 1 at d:\training\debug_demo01.py:21
(Pdb)
```

然后使用`continue`命令，会持续执行以下语句，知道遇到第一个断点：
```
(Pdb) continue
> d:\training\debug_demo01.py(21)afunc()
-> print(type(var), id(var), var)
(Pdb)
```

有时候，需要临时添加一个断点，例如在第11行语句：
```
        self.number = 0
```

可以使用`tbreak`来设置断点。其语法与`break`命令类似，但是执行一次后，会自动删除。
```
(Pdb) tbreak 11
Breakpoint 2 at d:\training\debug_demo01.py:11
(Pdb)
```

不跟选项，执行命令`break`会列出所有断点，包括临时断点：
```
(Pdb) b
Num Type         Disp Enb   Where
1   breakpoint   keep yes   at d:\training\debug_demo01.py:21
        breakpoint already hit 1 time
2   breakpoint   del  yes   at d:\training\debug_demo01.py:11
(Pdb)
```

设断点置后，可以使用命令`clear`来删除断点，其命令语法为：
- `clear`，清除所有断点；
- `clear filename:lineno`，清除指定文件行的断点；
- `clear bpnumber [bpnumber ...]`，清除指定断点。

例如，删除编号为1的断点：
```
(Pdb) cl 1
Deleted breakpoint 1 at d:\training\debug_demo01.py:21
(Pdb) b
Num Type         Disp Enb   Where
2   breakpoint   del  yes   at d:\training\debug_demo01.py:11
(Pdb)
```

使用`until`命令，并指定行数，可以持续执行直到运行到指定行（或遇到断点）。

也可以使用`jump`命令，也指定行数，同样持续执行直到运行到指定行（或遇到断点）。

这两个命令，可以更快捷运行到指定行数。

## 条件断点

在循环语句设置断点，那么每次循环都会在该断点中断。有时候需要在指定条件下才会中断，可以使用`condition`来设置条件断点。

例如，需要在第24行语句：
```
            result += i
```
设置条件断点，条件为`result == 3"时中断。

首先使用`break`设置断点，然后再使用`condition`命令设置条件：
```
(Pdb) b 24
Breakpoint 1 at d:\training\debug_demo01.py:24
(Pdb) condition 1 result == 3
New condition set for breakpoint 1.
(Pdb) b
Num Type         Disp Enb   Where
1   breakpoint   keep yes   at d:\training\debug_demo01.py:24
        stop only if result == 3
(Pdb)
```

条件断点设置完毕后，运行命令`continue`，检查是否符合条件：
```
(Pdb) c
<class 'int'> 1838444128 5
> d:\training\debug_demo01.py(24)afunc()
-> result += i
(Pdb) p result
3
(Pdb)
```

## 更改变量

在用pdb调试程序时，有时需要临时更改某个变量，以便尝试程序运行过程，可以在感叹号`!`后跟语句来改变变量。如果变量名不是pdb保留的命令，可以省略`!`。

例如在第21行语句：
```
    print(type(var), id(var), var)
```
程序传入的参数为整数5，我们可以更改为字符串`hello`，再来看看后续如何运行。

```
(Pdb) var = 'hello'
(Pdb) p var
'hello'
(Pdb) n
<class 'str'> 2213549452176 hello
> d:\training\debug_demo01.py(22)afunc()
-> if isinstance(var, int):
(Pdb)
> d:\training\debug_demo01.py(26)afunc()
-> return result
```

## 查看栈帧

使用pdb调试Python程序，开始时处于主程序中，当使用`step`命令，进入所调用函数体，也就是进入函数空间，可以用栈帧来表示。

**pdb有关栈帧的命令**

| 命令 | 说明  |
|:-----|:------|
| `args`, `a`| 列出当前函数的参数 |
| `bt` | 查看调用堆栈 |
| `where`| 打印当前执行堆栈 |
| `up`, `u`| 执行跳转到当前堆栈的上一层 |
| `down`, `d`| 执行跳转到在当前堆栈的深一层 |

例如，运行到第20行语句：
```
    result = 0
```

该语句是调用`main()`中的`afunc()`函数。

使用`args`命令查看函数参数：
```
(Pdb) c
> d:\training\debug_demo01.py(20)afunc()
-> result = 0
(Pdb) args
var = 5
(Pdb)
```

使用命令`bt`或`where`查看对应堆栈：
```
(Pdb) bt
  c:\anaconda3\lib\bdb.py(431)run()
-> exec(cmd, globals, locals)
  <string>(1)<module>()
  d:\training\debug_demo01.py(38)<module>()
-> main()
  d:\training\debug_demo01.py(31)main()
-> result = afunc(5)
> d:\training\debug_demo01.py(20)afunc()
-> result = 0
```

使用命令`up`回到上一层堆栈：
```
(Pdb) u
> d:\training\debug_demo01.py(31)main()
-> result = afunc(5)
(Pdb) where
  c:\anaconda3\lib\bdb.py(431)run()
-> exec(cmd, globals, locals)
  <string>(1)<module>()
  d:\training\debug_demo01.py(38)<module>()
-> main()
> d:\training\debug_demo01.py(31)main()
-> result = afunc(5)
  d:\training\debug_demo01.py(20)afunc()
-> result = 0
(Pdb)
```

## 小结

pdb对多线程，远程调试支持得不够好，没有使用图形界面，有时会觉得不太方便。可以使用PyCharm实现图形界面调试。